# Deploy MLflow Model to online endpoint

- Before running this notebook, run the **Chapter 6 Prep-Model Creation & Registration.ipynb** notebook to create and register a model for use

- Models created with MLFlow do not require a scoring script nor an environment

## In this notebook we will:

- Connect to your workspace.
- Create an online endpoint
- Retrieve and register a model from the job ran in the previous notebook
- Create a deployment
- Make an API Call to the managed online endpoint


In [71]:
import azure.ai.ml
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)

print(azure.ai.ml._version.VERSION)

1.1.0


In [72]:
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

In [73]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    # This will open a browser page for
    credential = InteractiveBrowserCredential()

In [74]:
#connect to the workspace
try:
    ml_client = MLClient.from_config(credential=credential)
except Exception as ex:
    # NOTE: Update following workspace information if not correctly configure before
    client_config = {
        "subscription_id": subscription_id,
        "resource_group": resource_group,
        "workspace_name": workspace,
    }

    if client_config["subscription_id"].startswith("<"):
        print(
            "please update your <SUBSCRIPTION_ID> <RESOURCE_GROUP> <AML_WORKSPACE_NAME> in notebook cell"
        )
        raise ex
    else:  # write and reload from config file
        import json, os

        config_path = "../.azureml/config.json"
        os.makedirs(os.path.dirname(config_path), exist_ok=True)
        with open(config_path, "w") as fo:
            fo.write(json.dumps(client_config))
        ml_client = MLClient.from_config(credential=credential, path=config_path)
print(ml_client)

Found the config file in: /mnt/batch/tasks/shared/LS_root/mounts/clusters/memasanz5/code/Users/memasanz/ML-Engineering-with-Azure-Machine-Learning-Service/.azureml/config.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7efffa1cbf70>,
         subscription_id=5da07161-3770-4a4b-aa43-418cbbb627cf,
         resource_group_name=mm-aml-mlops-dev-rg,
         workspace_name=mm-aml-mlops-dev)


## Create Online Endpoint

To create an online endpoint, we will leverage the class *ManagedOnlineEndpoint*.  To create the online endpoint we will provide the following configuration:

- name of endpoint
- description
- auth_mode (set to key) or aml_token
- tags - to provide information regarding the endpoint

### Configure endpoint

In [75]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "chp6-mlflow-endpt-" + datetime.datetime.now().strftime("%m%d%H%M%f")
print(len(online_endpoint_name))
# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="titanic online endpoint for mlflow model",
    auth_mode="key",
    tags={"mlflow": "true"},
)

32


## Create endpoint

Using the MLClient created earlier, we will now create the Endpoint in the workspace. This command will start the endpoint creation and return a confirmation response while the endpoint creation continues.


In [76]:
ml_client.begin_create_or_update(endpoint)

## Create deployment
A deployment is a set of resouces used for hosting the inferecing model using the *ManagedOnlineDeployment* class.  
Using the *ManagedOnlineDeployment* class, a developer can configure the following components

- name: name of the deployment
- endpoint_name: name of the endpoint to create the deployment under
- model: the model to use for the deployment
- instance_type: the VM side to use for deployment
- instance_count: the number of instances to use for the deployment
    

# Retrieve Model from registry

In [77]:
import mlflow

In [78]:
experiment = 'chapter6'
current_experiment=dict(mlflow.get_experiment_by_name(experiment))
experiment_id=current_experiment['experiment_id']
print(experiment_id)

df = mlflow.search_runs([experiment_id])

run_id = df['run_id'].tail(1).values[0]
print(run_id)

1f51d714-6c5b-47ea-9212-873cc346f796
busy_store_ly2m7jfdlw


In [79]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

run_model = Model(
    name="chapter6_titanic_model",
    path="azureml://jobs/" + run_id + "/outputs/artifacts/paths/model/",
    description="Model created from run.",
    type=AssetTypes.MLFLOW_MODEL 
)


In [80]:
run_model

Model({'job_name': None, 'is_anonymous': False, 'auto_increment_version': True, 'name': 'chapter6_titanic_model', 'description': 'Model created from run.', 'tags': {}, 'properties': {}, 'id': None, 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/memasanz5/code/Users/memasanz/ML-Engineering-with-Azure-Machine-Learning-Service/chapter 6', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f0007c3c700>, 'version': None, 'latest_version': None, 'path': 'azureml://jobs/busy_store_ly2m7jfdlw/outputs/artifacts/paths/model/', 'datastore': None, 'utc_time_created': None, 'flavors': None, 'arm_type': 'model_version', 'type': 'mlflow_model'})

## Configure the deployment

- retrieve the experiment id for this run, and the run id to retrieve the model from the registered model list

In [81]:
import time
while ml_client.online_endpoints.get(name=online_endpoint_name).provisioning_state == 'Creating':
    print('Creating')
    time.sleep(10)

print(ml_client.online_endpoints.get(name=online_endpoint_name).provisioning_state)

Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Succeeded


In [82]:
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=run_model,
    instance_type="Standard_F4s_v2",
    instance_count=1,
)

In [83]:
## Create deployment
print(online_endpoint_name)

chp6-mlflow-endpt-11302038545236


In [84]:
ml_client.online_deployments.begin_create_or_update(blue_deployment)

Check: endpoint chp6-mlflow-endpt-11302038545236 exists
data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


In [85]:
import time
while ml_client.online_deployments.get(name = "blue", endpoint_name = online_endpoint_name).provisioning_state == 'Updating':
    print('Updating..will take about 10 minutes to deploy...')
    time.sleep(5)
    
ml_client.online_deployments.get(name = "blue", endpoint_name = online_endpoint_name).provisioning_state

.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
..Updating..will take about 10 

'Succeeded'

.

In [86]:
## get status of online deployment
ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=online_endpoint_name, lines=50
)

"Instance status:\nSystemSetup: Succeeded\nUserContainerImagePull: Succeeded\nModelDownload: Succeeded\nUserContainerStart: Succeeded\n\nContainer events:\nKind: Pod, Name: ReadinessProbeFailed, Type: Warning, Time: 2022-11-30T20:53:47.288403Z, Message: Readiness probe failed: HTTP probe failed with statuscode: 502\nKind: Pod, Name: LivenessProbeFailed, Type: Warning, Time: 2022-11-30T20:53:52.091811Z, Message: Liveness probe failed: HTTP probe failed with statuscode: 502\nKind: Pod, Name: ReadinessProbeFailed, Type: Warning, Time: 2022-11-30T20:53:57.288271Z, Message: Readiness probe failed: HTTP probe failed with statuscode: 502\nKind: Pod, Name: LivenessProbeFailed, Type: Warning, Time: 2022-11-30T20:54:02.091947Z, Message: Liveness probe failed: HTTP probe failed with statuscode: 502\nKind: Pod, Name: ReadinessProbeFailed, Type: Warning, Time: 2022-11-30T20:54:07.288234Z, Message: Readiness probe failed: HTTP probe failed with statuscode: 502\nKind: Pod, Name: LivenessProbeFailed, 

In [87]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint)

## Checking provisioning state

In [88]:
import time
while ml_client.online_deployments.get(name = "blue", endpoint_name = online_endpoint_name).provisioning_state == 'Updating':
    print('Updating')
    time.sleep(5)
    
ml_client.online_deployments.get(name = "blue", endpoint_name = online_endpoint_name).provisioning_state

'Succeeded'

## Get Endpoint details

In [89]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(endpoint)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

primary_key = ml_client.online_endpoints.get_keys(name = online_endpoint_name).primary_key

print(primary_key)

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Updating', 'scoring_uri': 'https://chp6-mlflow-endpt-11302038545236.eastus.inference.ml.azure.com/score', 'openapi_uri': 'https://chp6-mlflow-endpt-11302038545236.eastus.inference.ml.azure.com/swagger.json', 'name': 'chp6-mlflow-endpt-11302038545236', 'description': 'titanic online endpoint for mlflow model', 'tags': {'mlflow': 'true'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/5da07161-3770-4a4b-aa43-418cbbb627cf/resourcegroups/mm-aml-mlops-dev-rg/providers/microsoft.machinelearningservices/workspaces/mm-aml-mlops-dev/onlineendpoints/chp6-mlflow-endpt-11302038545236', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/5da07161-3770-4a4b-aa43-418cbbb627cf/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oe:01f1a0ff-638e-4ff1-9168-5441f147de81:410c54a4-d056-4da6-bf74-3f4e78dbf443?api-version=2022-02-01-preview'}, 'id': '/subscriptions/5da0

### Leverage the registered ml table from Chapter 4 to get some data to send to the rest endpoint.

- Previously in Chapter 4, we registered the MLTable: titanic_prepped_mltable_x2, we will retrieve it and convert it to a pandas datafame

In [90]:
import mltable
registered_v1_data_asset = ml_client.data.get(name='titanic_prepped_mltable_x2', version='1')
print(registered_v1_data_asset.path)

tbl = mltable.load(uri=registered_v1_data_asset.path)
df = tbl.to_pandas_dataframe()

azureml://subscriptions/5da07161-3770-4a4b-aa43-418cbbb627cf/resourcegroups/mm-aml-mlops-dev-rg/workspaces/mm-aml-mlops-dev/datastores/workspaceblobstore/paths/LocalUpload/b3e9d2d76d36b52fc88b17546f0f0460/titanic_prepped_mltable/


In [91]:
columns_to_keep =  ['Embarked', 'Loc', 'Sex','Pclass', 'Age', 'Fare', 'GroupSize']
X_raw           = df[columns_to_keep]
X_raw

,Embarked,Loc,Sex,Pclass,Age,Fare,GroupSize
0,S,X,m,3,22.0,7.25,2
1,C,C,f,1,38.0,71.2833,2
2,S,X,f,3,26.0,7.925,1
3,S,C,f,1,35.0,53.1,2
4,S,X,m,3,35.0,8.05,1
...,...,...,...,...,...,...,...
886,S,X,m,2,27.0,13.0,1
887,S,B,f,1,19.0,30.0,1
888,S,X,f,3,21.5,23.45,4
889,C,C,m,1,26.0,30.0,1


In [93]:
import json
url = endpoint.scoring_uri
api_key = primary_key  # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'blue' }
import requests

def MakePrediction(df):
    strjson = str(df.to_json(orient='records'))
    endpoint_url = url

    request_data =  {
              "input_data": {
                "columns": [
                  "Embarked",
                  "Loc",
                  "Sex",
                  "Pclass",
                  "Age",
                  "Fare",
                  "GroupSize"
                ],
                "data": []
              }
            }

    request_df = X_raw.head(2)
    request_data['input_data']['data'] = json.loads(request_df.to_json(orient='records'))
    parsed = json.dumps(request_data)
    print(parsed)
    r = requests.post(endpoint_url, headers=headers, data=parsed)
    return (r.json())

results = MakePrediction(X_raw.head(2))
print(results)

{"input_data": {"columns": ["Embarked", "Loc", "Sex", "Pclass", "Age", "Fare", "GroupSize"], "data": [{"Embarked": "S", "Loc": "X", "Sex": "m", "Pclass": "3", "Age": "22.0", "Fare": "7.25", "GroupSize": "2"}, {"Embarked": "C", "Loc": "C", "Sex": "f", "Pclass": "1", "Age": "38.0", "Fare": "71.2833", "GroupSize": "2"}]}}
[0, 1]
